Used yolo8v to train the model with train dataset 1180 images.

Installig Ultralytics in Colab

In [1]:
!pip install ultralytics

**importing required libraries**

In [2]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Importing drive in colab

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Specifying file paths for a dataset organized in the context of training a YOLO8v model

In [4]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [5]:
# Split the dataset into train and test and creates the train.txt and test.tx with
# the respective path of the images in each folder


def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ## copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files

train_test_split('/content/drive/MyDrive/ui_data/valid/')

------ PROCESS STARTED -------
--- This folder has a total number of 1180 images---


  0%|          | 0/944 [00:00<?, ?it/s]

------ Training data created with 80% split 944 images -------


  0%|          | 0/236 [00:00<?, ?it/s]

------ Testing data created with a total of 236 images ----------
------ TASK COMPLETED -------


In [6]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.6 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.1/78.2 GB disk)


## Training

In [7]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/ui_data/data.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/ui_data/training_results name=web

100% 21.5M/21.5M [00:00<00:00, 167MB/s]
Ultralytics YOLOv8.1.6 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/ui_data/data.yaml, epochs=10, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/ui_data/training_results, name=web, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

## Inferencing

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/ui_data/training_results/web/weights/best.pt conf=0.55 source=/content/drive/MyDrive/ui_data/images

In [9]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/ui_data/output

In [12]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/ui_data/training_results/web/weights/best.pt conf=0.55 source=/content/drive/MyDrive/ui_data/app

Ultralytics YOLOv8.1.6 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11135646 parameters, 0 gradients, 28.5 GFLOPs

image 1/4 /content/drive/MyDrive/ui_data/app/Menu.png: 640x320 1 Card, 2 Icons, 3 Images, 8 Texts, 68.3ms
image 2/4 /content/drive/MyDrive/ui_data/app/Sign In Page.png: 640x320 1 Button, 3 Icons, 1 Image, 3 Texts, 10.6ms
image 3/4 /content/drive/MyDrive/ui_data/app/Sign Up Page.png: 640x320 1 Button, 3 Icons, 1 Image, 3 Texts, 10.6ms
image 4/4 /content/drive/MyDrive/ui_data/app/Splash Screen.png: 640x320 1 Text, 10.6ms
Speed: 1.9ms preprocess, 25.0ms inference, 155.5ms postprocess per image at shape (1, 3, 640, 320)
Results saved to runs/detect/predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [13]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/ui_data/output